In [ ]:
#importing openCV
import cv2

#importing numpy
import numpy as np

#importing pandas to read the CSV file containing our data
import pandas as pd

import datetime
import tensorflow as tf
import keras

#importing keras and sub-libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Conv2D, BatchNormalization, ELU
from tensorflow.keras import optimizers
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [ ]:
def extract_plate(img):
    
    height, width, channels = img.shape
    
    labelsPath = "./coco.names"
    weightsPath = "./custom-yolov4-detector_best.weights"
    configPath = "./custom-yolov4-detector.cfg"
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
    
    plate_img = img.copy()
    classes = []
    with open(labelsPath, "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layers_names = net.getLayerNames()
    output_layers = [layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392,size=(320,320), mean=(0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    boxes = []
    confs = []
    class_ids = []
    for output in outputs:
        for detect in output:
            scores = detect[5:]
            #print(scores)
            class_id = np.argmax(scores)
            conf = scores[class_id]
            if conf > 0.8:
                center_x = int(detect[0] * width)
                center_y = int(detect[1] * height)
                w = int(detect[2] * width)
                h = int(detect[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confs.append(float(conf))
                class_ids.append(class_id)
                
    return boxes, confs, class_ids

In [ ]:
def draw_labels(boxes, confs, colors, class_ids, classes, img): 
    indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    cv2.imshow("Image", img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [ ]:
def segment_characters(img):
    img1 = cv2.resize(img,(333,75))
    img_gray = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    _,img_binary = cv2.threshold(img_gray,200,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_erode = cv2.erode(img_binary,(3,3))
    img_dilate = cv2.dilate(img_erode,(3,3))
    
    LP_WIDTH = img_dilate.shape[0]
    LP_HEIGHT = img_dilate.shape[1]
    
    img_dilate[0:3,:] = 255
    img_dilate[:,0:3] = 255
    img_dilate[72:75,:] = 255
    img_dilate[:,330:333] = 255
    
    dimensions = [LP_WIDTH/6,LP_WIDTH/2,LP_HEIGHT/10,2*LP_HEIGHT/3]
    
    char_list = find_contours(dimensions,img_dilate)
    
    return char_list

In [ ]:
def find_contours(dimensions,img):
    cntrs,_ = cv2.findContours(img.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]
    
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]
    
    x_cntr_list = []
    target_contours = []
    img_res = []
    
    for contour in cntrs:
        intX,intY,intWidth,intHeight = cv2.boundingRect(contour)
        
        if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
            x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

            char_copy = np.zeros((44,24))
            #extracting each character using the enclosing rectangle's coordinates.
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            char = cv2.resize(char, (20, 40))

            # Make result formatted for classification: invert colors
            char = cv2.subtract(255, char)

            # Resize the image to 24x44 with black border
            char_copy[2:42, 2:22] = char
            char_copy[0:2, :] = 0
            char_copy[:, 0:2] = 0
            char_copy[42:44, :] = 0
            char_copy[:, 22:24] = 0

            #Reshaping 
            char_copy = cv2.resize(char_copy,(28,28))
            img_res.append(char_copy) #List that stores the character's binary image (unsorted)
            
    indices = sorted(range(len(x_cntr_list)),key=lambda k: x_cntr_list[k])
    
    img_res_copy = []
    
    for idx in indices:
        img_res_copy.append(img_res[idx])
    
    img_res = np.array(img_res_copy)
    
    return img_res

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16,kernel_size=(3,3),input_shape=(28,28,1),activation='relu'))
model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=48,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=36,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.00001),metrics=['accuracy'])

In [ ]:
'''from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.05,height_shift_range=0.05)

train_generator = train_datagen.flow_from_directory('data/train',target_size=(28,28),batch_size=1,color_mode='grayscale',class_mode='categorical')
validation_generator = train_datagen.flow_from_directory('data/val',target_size=(28,28),batch_size=1,color_mode='grayscale',class_mode='categorical')'''

Found 864 images belonging to 36 classes.
Found 216 images belonging to 36 classes.


In [ ]:
'''class stop_training_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if logs.get('val_acc') > 0.99:
            self.model.stop_training = True

log_dir = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

batch_size = 1
callbacks = [tensorboard_callback, stop_training_callback()]
model.fit_generator(train_generator,validation_data=validation_generator,epochs=100)'''

Epoch 1/100
864/864 [==============================] - 26s 30ms/step - loss: 3.5499 - acc: 0.0521 - val_loss: 3.4692 - val_acc: 0.2176
Epoch 2/100
864/864 [==============================] - 14s 16ms/step - loss: 3.3723 - acc: 0.2002 - val_loss: 3.1400 - val_acc: 0.4676
Epoch 3/100
864/864 [==============================] - 14s 16ms/step - loss: 2.8458 - acc: 0.4051 - val_loss: 2.3008 - val_acc: 0.6481
Epoch 4/100
864/864 [==============================] - 13s 15ms/step - loss: 1.8781 - acc: 0.6042 - val_loss: 1.3542 - val_acc: 0.7593
Epoch 5/100
864/864 [==============================] - 13s 15ms/step - loss: 1.2064 - acc: 0.6956 - val_loss: 0.8930 - val_acc: 0.8148
Epoch 6/100
864/864 [==============================] - 13s 14ms/step - loss: 0.8215 - acc: 0.7755 - val_loss: 0.6725 - val_acc: 0.8241
Epoch 7/100
864/864 [==============================] - 13s 15ms/step - loss: 0.7127 - acc: 0.7905 - val_loss: 0.5456 - val_acc: 0.8704
Epoch 8/100
864/864 [==============================] - 

In [ ]:
model.save("OCR_Model")

In [ ]:
def extractLPNumber(img):
    classifier = load_model("OCR_Model")
    alphabets = list('0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    result = extract_plate(img)
    lp_numbers = []
    for dim,_,cls in np.array(result).transpose():
        if cls == 0:
            x,y,w,h = dim
            segment = img[y:y+h,x:x+w]
            characters = segment_characters(segment)
            if len(characters)>0:
                string = ''
                pred = classifier.predict(characters.reshape((-1,28,28,1)))
                for output in pred:
                    #Appending the character
                    string += alphabets[np.argmax(output)]
                lp_numbers.append(string)
                
    return lp_numbers

In [ ]:
img = cv2.imread("img.jpg")
colors = np.random.uniform(0, 255, size=(3, 3))
boxes, confs, class_ids = extract_plate(img)
draw_labels(boxes,confs,colors,class_ids,["Number Plate","Car"],img)
img = cv2.imread("img.jpg")
extractLPNumber(img)

<ipython-input-135-f32b6c3287b7>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  for dim,_,cls in np.array(result).transpose():


['1KAP755', 'KA757']